In [1]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

from skforecast.recursive._forecaster_recursive_multiseries import ForecasterRecursiveMultiSeries
import skforecast
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection import grid_search_forecaster_multiseries

In [2]:
backtesting_forecaster_multiseries

<function skforecast.model_selection._validation.backtesting_forecaster_multiseries(forecaster: object, series: Union[pandas.core.frame.DataFrame, dict], cv: skforecast.model_selection._split.TimeSeriesFold, metric: Union[str, Callable, list], levels: Union[str, list, NoneType] = None, add_aggregated_metric: bool = True, exog: Union[pandas.core.series.Series, pandas.core.frame.DataFrame, dict, NoneType] = None, interval: Optional[list] = None, n_boot: int = 250, random_state: int = 123, use_in_sample_residuals: bool = True, n_jobs: Union[int, str] = 'auto', verbose: bool = False, show_progress: bool = True, suppress_warnings: bool = False) -> Tuple[pandas.core.frame.DataFrame, pandas.core.frame.DataFrame]>

In [3]:
# Data load
# ==============================================================================
train = pd.read_csv('data/train.csv')
val = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [4]:
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')

In [5]:
train = train.set_index('date')

In [6]:
train.head()

,id,country,store,product,num_sold
date,,,,,
2010-01-01,0,Canada,Discount Stickers,Holographic Goose,NaN
2010-01-01,1,Canada,Discount Stickers,Kaggle,973.0
2010-01-01,2,Canada,Discount Stickers,Kaggle Tiers,906.0
2010-01-01,3,Canada,Discount Stickers,Kerneler,423.0
2010-01-01,4,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [7]:
grouped = train.groupby(['country','store','product'])
grouped_dataframes = {key: group for key, group in grouped}

In [8]:
products = train.groupby(['product'])
products_dic = {key:group for key,group in products}
products_dic

{('Holographic Goose',):                 id    country                 store            product  \
 date                                                                     
 2010-01-01       0     Canada     Discount Stickers  Holographic Goose   
 2010-01-01       5     Canada     Stickers for Less  Holographic Goose   
 2010-01-01      10     Canada  Premium Sticker Mart  Holographic Goose   
 2010-01-01      15    Finland     Discount Stickers  Holographic Goose   
 2010-01-01      20    Finland     Stickers for Less  Holographic Goose   
 ...            ...        ...                   ...                ...   
 2016-12-31  230105     Norway     Stickers for Less  Holographic Goose   
 2016-12-31  230110     Norway  Premium Sticker Mart  Holographic Goose   
 2016-12-31  230115  Singapore     Discount Stickers  Holographic Goose   
 2016-12-31  230120  Singapore     Stickers for Less  Holographic Goose   
 2016-12-31  230125  Singapore  Premium Sticker Mart  Holographic Goose   
 

In [9]:
grouped_dataframes_selected = dict()
for key, dataset in grouped_dataframes.items():
    if dataset.isnull().any().sum() == 0:
        grouped_dataframes_selected[key] = dataset

In [10]:
final_df = pd.DataFrame({'_'.join(key):df['num_sold'] for key, df in grouped_dataframes_selected.items()})

In [11]:
final_df

,Canada_Discount Stickers_Kaggle,Canada_Discount Stickers_Kaggle Tiers,Canada_Discount Stickers_Kerneler Dark Mode,Canada_Premium Sticker Mart_Kaggle,Canada_Premium Sticker Mart_Kaggle Tiers,Canada_Premium Sticker Mart_Kerneler,Canada_Premium Sticker Mart_Kerneler Dark Mode,Canada_Stickers for Less_Kaggle,Canada_Stickers for Less_Kaggle Tiers,Canada_Stickers for Less_Kerneler,...,Singapore_Premium Sticker Mart_Holographic Goose,Singapore_Premium Sticker Mart_Kaggle,Singapore_Premium Sticker Mart_Kaggle Tiers,Singapore_Premium Sticker Mart_Kerneler,Singapore_Premium Sticker Mart_Kerneler Dark Mode,Singapore_Stickers for Less_Holographic Goose,Singapore_Stickers for Less_Kaggle,Singapore_Stickers for Less_Kaggle Tiers,Singapore_Stickers for Less_Kerneler,Singapore_Stickers for Less_Kerneler Dark Mode
date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,973.0,906.0,491.0,2212.0,2013.0,950.0,1213.0,1837.0,1659.0,807.0,...,317.0,2112.0,1992.0,1045.0,1255.0,301.0,1981.0,1565.0,841.0,1080.0
2010-01-02,881.0,854.0,437.0,2183.0,1953.0,895.0,1217.0,1972.0,1541.0,783.0,...,364.0,2322.0,1938.0,1010.0,1224.0,268.0,1763.0,1689.0,781.0,971.0
2010-01-03,1003.0,839.0,495.0,2459.0,1938.0,1015.0,1188.0,1936.0,1770.0,832.0,...,388.0,2253.0,2315.0,1040.0,1314.0,289.0,1801.0,1690.0,813.0,987.0
2010-01-04,744.0,609.0,441.0,1714.0,1567.0,802.0,1046.0,1382.0,1243.0,646.0,...,269.0,1760.0,1656.0,846.0,927.0,256.0,1543.0,1205.0,602.0,839.0
2010-01-05,707.0,640.0,372.0,1593.0,1590.0,773.0,894.0,1416.0,1319.0,612.0,...,264.0,1536.0,1410.0,770.0,962.0,226.0,1364.0,1352.0,629.0,795.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,694.0,577.0,372.0,1611.0,1316.0,775.0,935.0,1518.0,1104.0,612.0,...,351.0,2169.0,1906.0,1079.0,1179.0,267.0,1948.0,1483.0,853.0,1016.0
2016-12-28,810.0,601.0,424.0,1812.0,1418.0,802.0,927.0,1702.0,1249.0,733.0,...,371.0,2558.0,1742.0,1050.0,1177.0,316.0,2127.0,1580.0,962.0,1062.0
2016-12-29,856.0,687.0,442.0,2157.0,1604.0,937.0,1110.0,1607.0,1137.0,746.0,...,406.0,2833.0,2238.0,1133.0,1421.0,357.0,2269.0,1788.0,980.0,1178.0


In [12]:
products = ['Holographic Goose', 'Kaggle','Kaggle Tiers', 'Kerneler Dark Mode', 'Kerneler']

In [13]:
prod_df_dic = dict()
for prod in products:
    prod_df = pd.DataFrame()
    for col in final_df.columns:
        if prod in col:
            prod_df = pd.concat([prod_df, final_df[col]], axis=1)
            prod_df_dic[prod] = prod_df

In [14]:
prod_df_dic['Holographic Goose']

,Finland_Discount Stickers_Holographic Goose,Finland_Premium Sticker Mart_Holographic Goose,Finland_Stickers for Less_Holographic Goose,Italy_Discount Stickers_Holographic Goose,Italy_Premium Sticker Mart_Holographic Goose,Italy_Stickers for Less_Holographic Goose,Norway_Discount Stickers_Holographic Goose,Norway_Premium Sticker Mart_Holographic Goose,Norway_Stickers for Less_Holographic Goose,Singapore_Discount Stickers_Holographic Goose,Singapore_Premium Sticker Mart_Holographic Goose,Singapore_Stickers for Less_Holographic Goose
2010-01-01 00:00:00,150.0,362.0,311.0,110.0,269.0,228.0,261.0,625.0,579.0,150.0,317.0,301.0
2010-01-02 00:00:00,127.0,297.0,287.0,107.0,251.0,226.0,274.0,643.0,475.0,146.0,364.0,268.0
2010-01-03 00:00:00,152.0,357.0,308.0,122.0,266.0,236.0,267.0,654.0,568.0,155.0,388.0,289.0
2010-01-04 00:00:00,113.0,295.0,239.0,91.0,213.0,187.0,228.0,496.0,445.0,123.0,269.0,256.0
2010-01-05 00:00:00,116.0,290.0,227.0,77.0,225.0,180.0,206.0,478.0,400.0,111.0,264.0,226.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27 00:00:00,117.0,279.0,230.0,75.0,185.0,170.0,175.0,435.0,404.0,153.0,351.0,267.0
2016-12-28 00:00:00,130.0,344.0,315.0,81.0,206.0,172.0,186.0,509.0,409.0,158.0,371.0,316.0
2016-12-29 00:00:00,138.0,310.0,293.0,103.0,219.0,193.0,231.0,498.0,388.0,194.0,406.0,357.0
2016-12-30 00:00:00,138.0,338.0,303.0,99.0,239.0,199.0,223.0,541.0,468.0,184.0,420.0,359.0


In [15]:
end_train = '2015-06-17'
final_df_train = final_df.loc[:end_train].copy()
final_df_test = final_df.loc[end_train:].copy()

print(
    f"Train dates : {final_df_train.index.min()} --- {final_df_train.index.max()}   "
    f"(n={len(final_df_train)})"
)
print(
    f"Test dates  : {final_df_test.index.min()} --- {final_df_test.index.max()}   "
    f"(n={len(final_df_test)})"
)

Train dates : 2010-01-01 00:00:00 --- 2015-06-17 00:00:00   (n=1994)
Test dates  : 2015-06-17 00:00:00 --- 2016-12-31 00:00:00   (n=564)


In [16]:
final_df_train

,Canada_Discount Stickers_Kaggle,Canada_Discount Stickers_Kaggle Tiers,Canada_Discount Stickers_Kerneler Dark Mode,Canada_Premium Sticker Mart_Kaggle,Canada_Premium Sticker Mart_Kaggle Tiers,Canada_Premium Sticker Mart_Kerneler,Canada_Premium Sticker Mart_Kerneler Dark Mode,Canada_Stickers for Less_Kaggle,Canada_Stickers for Less_Kaggle Tiers,Canada_Stickers for Less_Kerneler,...,Singapore_Premium Sticker Mart_Holographic Goose,Singapore_Premium Sticker Mart_Kaggle,Singapore_Premium Sticker Mart_Kaggle Tiers,Singapore_Premium Sticker Mart_Kerneler,Singapore_Premium Sticker Mart_Kerneler Dark Mode,Singapore_Stickers for Less_Holographic Goose,Singapore_Stickers for Less_Kaggle,Singapore_Stickers for Less_Kaggle Tiers,Singapore_Stickers for Less_Kerneler,Singapore_Stickers for Less_Kerneler Dark Mode
date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,973.0,906.0,491.0,2212.0,2013.0,950.0,1213.0,1837.0,1659.0,807.0,...,317.0,2112.0,1992.0,1045.0,1255.0,301.0,1981.0,1565.0,841.0,1080.0
2010-01-02,881.0,854.0,437.0,2183.0,1953.0,895.0,1217.0,1972.0,1541.0,783.0,...,364.0,2322.0,1938.0,1010.0,1224.0,268.0,1763.0,1689.0,781.0,971.0
2010-01-03,1003.0,839.0,495.0,2459.0,1938.0,1015.0,1188.0,1936.0,1770.0,832.0,...,388.0,2253.0,2315.0,1040.0,1314.0,289.0,1801.0,1690.0,813.0,987.0
2010-01-04,744.0,609.0,441.0,1714.0,1567.0,802.0,1046.0,1382.0,1243.0,646.0,...,269.0,1760.0,1656.0,846.0,927.0,256.0,1543.0,1205.0,602.0,839.0
2010-01-05,707.0,640.0,372.0,1593.0,1590.0,773.0,894.0,1416.0,1319.0,612.0,...,264.0,1536.0,1410.0,770.0,962.0,226.0,1364.0,1352.0,629.0,795.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-06-13,799.0,573.0,386.0,1825.0,1410.0,663.0,870.0,1431.0,1173.0,626.0,...,235.0,2051.0,1665.0,886.0,1191.0,220.0,1743.0,1495.0,832.0,997.0
2015-06-14,811.0,629.0,411.0,1707.0,1476.0,714.0,1062.0,1647.0,1234.0,637.0,...,289.0,2502.0,1700.0,1028.0,1276.0,227.0,1885.0,1740.0,827.0,1186.0
2015-06-15,626.0,473.0,358.0,1484.0,1216.0,675.0,861.0,1282.0,1110.0,547.0,...,226.0,1926.0,1361.0,830.0,1093.0,187.0,1536.0,1453.0,687.0,807.0


In [17]:
# Create and fit forecaster multi series
# ==============================================================================
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = (random_state=123),
                 lags               = 564,
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None
             )

forecaster.fit(series=final_df_train)
forecaster


c:\Users\josin\anaconda3\envs\skforecast_env\Lib\site-packages\skforecast\recursive\_forecaster_recursive_multiseries.py:439: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(
c:\Users\josin\anaconda3\envs\skforecast_env\Lib\site-packages\skforecast\utils\utils.py:1157: UserWarning: Series has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(


============================== 
ForecasterRecursiveMultiSeries 
============================== 
Regressor: Ridge 
Lags: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360
 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378
 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396
 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414
 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432
 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450
 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468
 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486
 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504
 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522
 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540
 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558
 559 560 561 562 563 564] 
Window features: None 
Window size: 564 
Series encoding: ordinal 
Series names (levels): 
    Canada_Discount Stickers_Kaggle, Canada_Discount Stickers_Kaggle Tiers,
    Canada_Discount Stickers_Kerneler Dark Mode, Canada_Premium Sticker
    Mart_Kaggle, Canada_Premium Sticker Mart_Kaggle Tiers, Canada_Premium
    Sticker Mart_Kerneler, Canada_Premium Sticker Mart_Kerneler Dark Mode,
    Canada_Stickers for Less_Kaggle, Canada_Stickers for Less_Kaggle Tiers,
    Canada_Stickers for Less_Kerneler, Canada_Stickers for Less_Kerneler Dark
    Mode, Finland_Discount Stickers_Holographic Goose, Finland_Discount
    Stickers_Kaggle, Finland_Discount Stickers_Kaggle Tiers, Finland_Discount
    Stickers_Kerneler, Finland_Discount Stickers_Kerneler Dark Mode,
    Finland_Premium Sticker Mart_Holographic Goose, Finland_Premium Sticker
    Mart_Kaggle, Finland_Premium Sticker Mart_Kaggle Tiers, Finland_Premium
    Sticker Mart_Kerneler, Finland_Premium Sticker Mart_Kerneler Dark Mode,
    Finland_Stickers for Less_Holographic Goose, Finland_Stickers for
    Less_Kaggle, Finland_Stickers for Less_Kaggle Tiers, Finland_Stickers for
    Less_Kerneler, ..., Norway_Premium Sticker Mart_Holographic Goose,
    Norway_Premium Sticker Mart_Kaggle, Norway_Premium Sticker Mart_Kaggle
    Tiers, Norway_Premium Sticker Mart_Kerneler, Norway_Premium Sticker
    Mart_Kerneler Dark Mode, Norway_Stickers for Less_Holographic Goose,
    Norway_Stickers for Less_Kaggle, Norway_Stickers for Less_Kaggle Tiers,
    Norway_Stickers for Less_Kerneler, Norway_Stickers for Less_Kerneler Dark
    Mode

In [18]:
# Predict and predict_interval
# ==============================================================================
steps = 564

# Predictions for item_1
predictions_item_1 = forecaster.predict(steps=steps, levels='Canada_Discount Stickers_Kaggle')
display(predictions_item_1.head(5))

,Canada_Discount Stickers_Kaggle
1994,657.781253
1995,693.992923
1996,742.703195
1997,804.706850
1998,665.230720


In [19]:
type(predictions_item_1)

pandas.core.frame.DataFrame

In [20]:
error_mae = mean_absolute_error(final_df_test['Canada_Discount Stickers_Kaggle'], predictions_item_1)
error_rmse = root_mean_squared_error(
                y_true = final_df_test['Canada_Discount Stickers_Kaggle'],
                y_pred = predictions_item_1
            )
error_mape = mean_absolute_percentage_error(
                y_true = final_df_test['Canada_Discount Stickers_Kaggle'],
                y_pred = predictions_item_1
            )

In [21]:
print('mae: ', error_mae)
print('rmse: ', error_rmse)
print('mape: ', error_mape)

mae:  80.0536624141604
rmse:  91.9966675552214
mape:  0.12661847859062314


In [22]:
# Backtesting Multi Series
# ==============================================================================
metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                           forecaster         = forecaster,
                                           series             = final_df,
                                           levels             = None,
                                           #steps              = 564,
                                           metric             = 'mean_absolute_error',
                                           initial_train_size = len(final_df_train),
                                           refit              = True,
                                           fixed_train_size   = True,
                                           verbose            = False
                                       )

print("Backtest metrics")
display(metrics_levels)
print("")
print("Backtest predictions")
backtest_predictions.head(4)

TypeError: backtesting_forecaster_multiseries() got an unexpected keyword argument 'initial_train_size'